Try to use these to fix outputs issue:

https://ipython.readthedocs.io/en/stable/interactive/magics.html
https://notebook.community/lifeinoppo/littlefishlet-scode/RES/REF/python_sourcecode/ipython-master/examples/IPython%20Kernel/Capturing%20Output

# Common libs

notebook --allow-root --no-browser --notebook-dir=/root/model/

In [ ]:
# noinspection PyUnresolvedReferences
from importlib import reload

MODEL_PARAMS_DIR = "_model_results/model_params__mine"
RUNS_DIR = "_model_results/runs__mine"

# Load My Dataset

### Load RAW EEG for prediction

In [ ]:
import dataset_processing.my_dataset_loader

reload(dataset_processing.my_dataset_loader)

from dataset_processing.my_dataset_loader import EpocDatasetLoader

sampling_frequency = 200  # 200 Hz

_loader = EpocDatasetLoader(
    target_fs=sampling_frequency,
    seconds_per_eeg_for_gan=15,
    my_eeg_dir="datasets/_DEMO__EPOC_X/EEG",
)
_eeg_data_df = _loader.process_dataset()

# 200 Hz * 15 seconds -> 3000 num_samples
print("Dataset size:", len(_eeg_data_df.index))
print("EEG Shape:", _eeg_data_df.iloc[0]["EEG"].shape)

In [ ]:
# Segment into 1s segments
_eeg_data_df = _loader.segment_df_final(_eeg_data_df)
print("Dataset size:", len(_eeg_data_df.index))
print("EEG Shape:", _eeg_data_df.iloc[-1]["EEG"].shape)

### Data Augmentation

In [ ]:
from dataset_processing.eeg_augmentation import EEGAugmentation

_augmentor = EEGAugmentation(_eeg_data_df)
_augmented_df = _augmentor.augment_data()
del _augmentor

# Pre-Training

In [ ]:
from model.pre_training.do_pre_training import PreTraining

PRETRAINING_SAVE_DIR = f"{MODEL_PARAMS_DIR}/pretraining"
PRETRAINING_RUNS_DIR = f"{RUNS_DIR}/pretraining"

In [ ]:
with_gan = False

PRETRAINING_EXTRA = "WITH_GAN" if with_gan else "WITHOUT_GAN"
FINETUNING_EXTRA = "pre_GAN" if with_gan else "pre_no_GAN"

pretraining_model = PreTraining(
    data_loader=None,
    sampling_frequency=sampling_frequency,
    pretraining_model_save_dir=f"{PRETRAINING_SAVE_DIR}_{PRETRAINING_EXTRA}",
    scheduler_patience=50,
    early_stopping_patience=100,
    epochs=2000,
    to_train=False,
)
pretraining_model.load_model(2000)

# Fine-Tuning

In [ ]:
from torch.utils.data import DataLoader
from dataset_processing.eeg_dataset import EEGDataset
from model.fine_tuning.do_fine_tuning import FineTuning

FINETUNING_SAVE_DIR = f"{MODEL_PARAMS_DIR}/finetuning"
FINETUNING_RUNS_DIR = f"{RUNS_DIR}/finetuning"

# From the paper
finetuning_batch_size = 128
finetuning_epochs = 20

### Dataset Loader

In [ ]:
verdict_to_str = {
    -1: "NEGATIVE",
    0: "NEUTRAL",
    1: "POSITIVE",
    None: "OVERALL",
}

In [ ]:
def create_verdict_dataloader(verdict, num_workers=5, prefetch_factor=2, exclude_p=0):
    if verdict is None:
        filtered_df = _augmented_df
    else:
        filtered_df = _augmented_df[_augmented_df["Verdict"] == verdict]

    if exclude_p > 0:
        dataset_exclude_count = int(len(filtered_df.index) * exclude_p)
        filtered_df = filtered_df.iloc[dataset_exclude_count:-dataset_exclude_count]

    dataloader = DataLoader(
        EEGDataset(filtered_df),
        # batch_size=finetuning_batch_size,
        batch_size=128,
        shuffle=False,
        pin_memory=True,
        persistent_workers=True,
        num_workers=num_workers,
        prefetch_factor=prefetch_factor,
    )
    print(f"Verdict: {verdict}, Eval size: {len(filtered_df)}")
    return dataloader


verdicts = sorted(_augmented_df["Verdict"].unique())
dataloaders = {
    verdict: create_verdict_dataloader(verdict)
    for verdict in verdicts
}
dataloaders[None] = create_verdict_dataloader(None)

### Do the fine-tuning

In [ ]:
import model.fine_tuning.do_fine_tuning

reload(model.fine_tuning.do_fine_tuning)

from model.fine_tuning.do_fine_tuning import FineTuning
from utils.misc import print_accuracy_progress_bars, get_pred_str


# Load model where the maximum accuracy was achieved
best_model_type = "pre_no_GAN_WITH_GAN"
best_epoch = 5

for verdict, data_loader in dataloaders.items():
    finetuning_model = FineTuning(
        data_loader=None,
        data_loader_eval=data_loader,
        sampling_frequency=sampling_frequency,
        num_classes=3,
        num_channels=14,

        ET=pretraining_model.ET,
        EF=pretraining_model.EF,
        PT=pretraining_model.PT,
        PF=pretraining_model.PF,

        finetuning_model_save_dir=f"{FINETUNING_SAVE_DIR}_{best_model_type}",
        to_train=False,
    )
    finetuning_model.load_model(best_epoch, allow_logs=False)

    all_predictions, all_ok_predictions, eval_accuracy, avg_eval_loss = finetuning_model.do_eval_epoch(all_accuracies=True)
    print(
        f"Verdict {verdict_to_str[verdict]} -> "
        f"Overall Predictions: {get_pred_str(all_predictions, verdict_to_str)}"
    )

    print_accuracy_progress_bars(all_predictions, verdict_to_str, total_bar_length=30)

In [ ]:
print("DONE!")